# IKATS Python API

This Notebook is a tutorial about how to use the **IKATS Python API**.

In [1]:
# Import IKATS Api
from ikats import IkatsAPI
from ikats.exceptions import *
from ikats.extra.timeseries import *

# Create the API instance connected to localhost
if False:
    api = IkatsAPI(host="http://preprod.intra.ikats.org", port=80)
    api.session.tdm_url = "/dev-datamodel"
    api.session.tsdb_url = "/dev-opentsdb"
else:
    api = IkatsAPI(host="http://localhost", port=80)

## Timeseries

This part shows how to :
- create a new Timeseries
- add points to it (using a random points generator)
- save it
- delete it

In [2]:
# Create a local timeseries
ts = api.ts.new()
print("New TS locally created:", ts)
print("No TSUID is defined:", ts.tsuid)
print("No FID is defined:", ts.fid)
print("No points:", ts.data)



New TS locally created: <LocalTimeseries>
No TSUID is defined: None
No FID is defined: None
No points: []


In [5]:
# Adding some random points
ts.data = gen_random_ts(sd=1000000000000, ed=1000000010000, nb_points=10)

# Set a FID
ts.fid = "MyTS"
print("A FID is added:", ts.fid)
print("but still no TSUID:",ts.tsuid)
print("and the points are still there:", len(ts)) # You could also use `len(ts.data)`

A FID is added: MyTS
but still no TSUID: None
and the points are still there: 10


In [6]:
# Now we save the TS and request to generate minimum metadata (ikats_start_date, ikats_end_date, qual_nb_points)
ts.save(generate_metadata=True)

# Check what's new
print("The TSUID is now set to %s (because of the save action)" % ts.tsuid)

The TSUID is now set to 13E3260000010006E10000020006E00000030006E2 (because of the save action)


In [10]:
# We can still add points locally
ts.data.extend(gen_random_ts(sd=1000000011000, ed=1000000020000, nb_points=10))
print("The length of the TS has increased : %s" % len(ts))

# But we need to save them.
ts.save()

The length of the TS has increased : 30


In [15]:
# Deletion of the Timeseries
if ts.delete(raise_exception=False):
    print("The Timeseries has been deleted")

try:
    ts.delete(raise_exception=True)
except:
    # not working : see bugs #2738
    print("The Timeseries has already been deleted (first way to check it)")
    
if ts.delete(raise_exception=False):
    print("The Timeseries has already been deleted (second way to check it)")
    
# Timeseries still locally present. Save it again (for other cases to cover)
ts.save()

The Timeseries has been deleted
The Timeseries has already been deleted (second way to check it)


## Metadata

In [20]:
from ikats.objects.metadata_ import DTYPE

# Add specific metadata locally (corresponding to "generate_metadata" flag of `ts.save()`)
ts.metadata.set(name="qual_nb_points", value=len(ts), dtype=DTYPE.NUMBER)
ts.metadata.set(name="ikats_start_date", value=ts.data[0][0], dtype=DTYPE.NUMBER)
ts.metadata.set(name="ikats_end_date", value=ts.data[-1][0], dtype=DTYPE.NUMBER)

# Read metadata from local cache
print ("qual_nb_points=%d"%ts.metadata.get("qual_nb_points"))

# Save metadata
ts.metadata.save()

# Delete metadata from local cache
ts.metadata.set(name="test_md", value="42", dtype=DTYPE.STRING)
ts.metadata.delete(name="test_md")
try:
    ts.metadata.get(name="test_md")
except ValueError:
    print("This metadata doesn't exist")

try:
    ts.metadata.delete(name="test_md")
except IkatsNotFoundError:
    print("This metadata doesn't exist")



qual_nb_points=30
This metadata doesn't exist


## Dataset

In [ ]:
# Get the list of all Datasets
api.ds.list()

In [ ]:
# Create a Dataset
ds = api.ds.new(name="MyDataset")

In [ ]:
# Read a Dataset
print("Getting Dataset '%s'" % ds.name)
print("The description of the Dataset is '%s'" % ds.desc)
print("The Dataset is composed of %s Timeseries:  " % len(ds)) # You could also use `len(ds.ts)`
for ts in ds.ts:
    print("  - %s" % ts)

In [ ]:
# Save as a new Dataset
ds.name = "New_Name"
ds.description = "My new description"
try:
    api.ds.save(ds)
    print("Dataset saved")
except IkatsConflictError:
    print("Dataset already exist")

In [ ]:
# Delete the dataset
ds.name = "New_Name"
try:
    api.ds.delete(ds.name)
    print("Dataset deleted")
except IkatsNotFoundError:
    print("Dataset Not Found")

In [ ]:
api.ts.tsdb_client._get_metric_tags_from_tsuid('4C08320000010006E60000020006E0000003000666')